In [6]:
from openai import OpenAI
import base64
import requests

In [7]:
# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

In [8]:
client = OpenAI(
    api_key="sk-aitunnel-lRVzfpdyQrVrYJGyxiFkgMMXLSz5P2Oz", # Ключ из нашего сервиса
    base_url="https://api.aitunnel.ru/v1/",
)

In [9]:
def predict(prompt, image_paths=[], model="gpt-4o"):
    content = [
        {
            "type": "text",
            "text": prompt
        }
    ]
    
    if len(image_paths) > 0:
        for image_path in image_paths:
            base64_image = encode_image(image_path)
            content.append({
                "type": "image_url",
                "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}
            })
    
    messages=[
        {
            "role": "user",
            "content": content
        }
    ]
    completion = client.chat.completions.create(
        messages=messages,
        max_tokens=500,
        model=model
    )

    text_outputs = completion.choices[0].message.content
    return text_outputs

In [19]:
def foo(text, model="gpt-4o", **kwargs):
    print(text)
    print(model)
    print(kwargs)

foo("Hello", model="kekw")

Hello
kekw
{}


In [11]:
image_path1 = "/home/mpatratskiy/work/meta_world/llserver/notebooks/metatmp1.png"
image_path2 = "/home/mpatratskiy/work/pybullet/Pybullet/saved_images/image.png"

prompt = "Опиши эти изображения, что у них общего и чем они отличаются"
image_paths = [image_path1, image_path2]
model = "gemini-pro-1.5"

answer = predict(prompt, image_paths=image_paths, model=model)
print(answer)

Оба изображения демонстрируют роботизированную руку, манипулирующую объектами, но в разных условиях и с разными задачами.

**Общее:**

* **Роботизированный манипулятор:** На обоих изображениях присутствует роботизированная рука, предназначенная для манипулирования объектами. Хоть руки и выглядят немного разными (особенно захват), базовая функция та же.
* **Наличие объектов для манипуляции:** На обоих изображениях есть объекты, с которыми взаимодействует робот (шарики, кубики).
* **Цель — манипуляция:** Основная цель в обоих случаях — использовать роботизированную руку для перемещения или взаимодействия с объектами определенным образом.

**Различия:**

* **Среда:** Первое изображение показывает робота, работающего над столом или платформой, похожей на игровую доску.  Второе изображение показывает робота, работающего на плоской черной поверхности, напоминающей коврик. 
* **Тип объекта:** На первом изображении робот взаимодействует с шариками и одним кубиком. На втором — с кубиками и небо

In [16]:
image_path = "/home/mpatratskiy/work/meta_world/llserver/notebooks/metatmp1.png"
base64_image = encode_image(image_path)

completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "What’s in this image?"
                },
                {
                    "type": "image_url",
                    "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}
                }
            ]
        }
    ],
    max_tokens=500, # Старайтесь указывать для более точного расчёта цены
    model="gemini-pro-1.5"
)

print(completion.choices[0].message.content)

ChatCompletionMessage(content='The image shows a simulated robotic arm interacting with objects in a simulated environment. The arm, primarily red, is positioned over a rectangular wooden surface, possibly a table or platform.\n\nHere\'s a breakdown of the elements:\n\n* **Robotic Arm:** The main subject is a multi-jointed robotic arm in a predominantly red color. It appears to be a collaborative robot (cobot) designed for interaction with a shared workspace. The arm is extended over the table, and the end-effector (the "hand" of the robot) seems to have a two-fingered gripper or a similar tool attached.\n* **Table/Platform:** The wooden surface below the robotic arm is set within a metal or plastic frame, creating a container-like structure. This suggests a controlled environment for the robot\'s tasks.\n* **Objects:**\n    * A small red cylinder is on the wooden surface.  It is likely the target object for the robotic arm.\n    * A small blue sphere is visible near the back edge of t

In [ ]:


# Path to your image
image_path = "path_to_your_image.jpg"

# Getting the base64 string


headers = {
  "Content-Type": "application/json",
  "Authorization": f"Bearer {api_key}"
}

payload = {
  "model": "gpt-4o",
  "messages": [
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "What’s in this image?"
        },
        {
          "type": "image_url",
          "image_url": {
            "url": f"data:image/jpeg;base64,{base64_image}"
          }
        }
      ]
    }
  ],
  "max_tokens": 300
}

response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

print(response.json())

In [1]:
text = """
You are a helpful embodied assistant. You are helping with resolving errors in robot execution.
During execution of last action you have encountered a problem. Robot failed executing last action.
You are given a goal and a current plan and description of the current scene with ideas of what could be wrong.
Provde ideas how to fix the problem. HINT: robot may accidently drop objects during gripper movement but continue to move fripper with object.
Predict new plan what can fix the problem and reach the goal. Explain each choosen action briefly. You must use actions only from avaliable_actions in you predicted plan. Pay attention on action description. Do not use any other actions and formatting. Do not create combinations of actions.

Action description:
locate('object') - function that give you position of the object in the scene. Allows to move robot arm to the object in order to pick it up or place_on_top_of another object on it.
pick('object') - function that pick up an object. Can be used only if object location is known. Thus you must use locate('object') before pick('object').
place_on_top_of('object') - function that put something from gripper on top of the object, which name is provided to function. Example: place_on_top_of('blue block') - put anything from gripper on top of the blue block.
done() - function that indicate that the task is completed.
Each of locate, pick and place_on_top_of actions takes one argument - object name. You can not use any other arguments or multiple objects.

Strong advice: 
- always use locate('object') before pick('object')
- always use locate('object') before place_on_top_of('object')
- pick('object') can be used only if gripper is empty and object is located
- place_on_top_of('object') can be used only if gripper is NOT empty and object is located

Scene description and ideas of what could be wrong:
T

Avaliable actions:
["locate('red bowl')", "locate('red block')", "locate('green block')", "place_on_top_of('red bowl')", "place_on_top_of('blue block')", "place_on_top_of('green block')", "pick('blue block')", "locate('blue block')", 'done()', "pick('red bowl')", "pick('red block')", "pick('green block')", "place_on_top_of('red block')"]

Successfully executed actions:
["locate('blue block')", "pick('blue block')", "locate('red bowl')"]

Current plan:
["place_on_top_of('red bowl')", "locate('red block')", "pick('red block')", "locate('blue block')", "place_on_top_of('blue block')", "locate('green block')", "pick('green block')", "locate('red block')", "place_on_top_of('red block')", 'done()']

Current goal:
stack all blocks on each other in the red bowl
""".split()
len(text)

369